# **Mounting google drive**

In [ ]:
# Execute only in case of using Google Colab
# from google.colab import drive
# drive.mount('/content/drive')

# **Importing libraries**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import timeit
import hashlib
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import gc
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [4]:
df_amenities = pd.read_csv('path/amenities.tsv.gz', compression='gzip', sep='\t')

In [ ]:
# Load dataset with tqdm progress bar
with tqdm(total=1, desc="Loading dataset") as pbar:
   df2 = pd.read_csv('path/main.tsv.gz', compression='gzip', sep='\t')
   pbar.update(1)

print("Dataset loaded successfully!")
df=df2.copy()

Converting search id and user id from strings to unique ids

# Function to convert string to integer ID
def convert_to_int_id(string):
    # Convert string to bytes and hash it using MD5
    hash_object = hashlib.md5(string.encode())
    # Convert hexadecimal digest to integer
    int_id = int(hash_object.hexdigest(), 16)
    return int_id


df['search_id'] = pd.factorize(df['search_id'].apply(convert_to_int_id))[0]
df['user_id'] = pd.factorize(df['user_id'].apply(convert_to_int_id))[0]



In [ ]:
# Function to convert string to integer ID
def convert_to_int_id(string):
    # Convert string to bytes and hash it using MD5
    hash_object = hashlib.md5(string.encode())
    # Convert hexadecimal digest to integer
    int_id = int(hash_object.hexdigest(), 16)
    return int_id


df['search_id'] = pd.factorize(df['search_id'].apply(convert_to_int_id))[0]
df['user_id'] = pd.factorize(df['user_id'].apply(convert_to_int_id))[0]



# **Introducing new features**

- length of stay (checkout_date-checkin_date)
- booking window (day of year(checkin)-day of year(serch_timestamp))
- search_day_of_week= df['search_timestamp'].dt.dayofweek
- search_hour_of_day= df['search_timestamp'].dt.hour

In [ ]:
df['length_of_stay']=(pd.to_datetime(df['checkout_date'])-pd.to_datetime(df['checkin_date'])).dt.days
df['booking_window'] = ((pd.to_datetime(df['checkin_date'], utc=True))  - pd.to_datetime(df['search_timestamp'], utc=True)).dt.days
df['booking_window'] = df['booking_window'].apply(lambda x: 0 if x < 0 else x) # ensures that all negative values in the 'booking_window' column are set to 0.
df.dropna(subset=['booking_window', 'checkin_date', 'checkout_date'], inplace=True)# drop queries where no information on checkin_date and checkout_date
df['search_timestamp']=pd.to_datetime(df['search_timestamp'])
df['search_day_of_week'], df['search_hour_of_day']=df['search_timestamp'].dt.dayofweek, df['search_timestamp'].dt.hour
df['checkin_day']=pd.to_datetime(df['checkin_date']).dt.dayofweek
df['checkout_day']=pd.to_datetime(df['checkin_date']).dt.dayofweek



In [ ]:
df.drop(columns=['user_id','search_timestamp','applied_filters','room_count','infant_count'],inplace=True)

Let's say we have the following data:

checkin_date: 2021-06-01
checkout_date: 2021-06-05
The expression (df['checkin_date'] + pd.to_timedelta(range((df['checkout_date'] - df['checkin_date']).days), 'D')).dt.dayofweek calculates the day of the week for each day of the stay (from check-in to check-out) in the given date range.

Here's how it works step by step:

Calculate the duration of the stay: (df['checkout_date'] - df['checkin_date']).days, which gives us 4 days (from June 1st to June 5th, inclusive).
Generate a range of timedelta objects for each day of the stay: range(4) gives us [0, 1, 2, 3].
Add these timedelta objects to the check-in date: df['checkin_date'] + pd.to_timedelta(range(4), 'D') gives us the dates [2021-06-01, 2021-06-02, 2021-06-03, 2021-06-04].
Extract the day of the week for each date: .dt.dayofweek gives us [1, 2, 3, 4], where 0 represents Monday, 1 represents Tuesday, and so on.
So, the final output would be [1, 2, 3, 4], indicating that the stay covers Tuesday, Wednesday, Thursday, and Friday.

# **Explode of impression columns**

In [ ]:
df=df[df['sort_type']=='RECOMMENDED']

In [ ]:
df.drop(columns=['sort_type'],inplace=True)

In [ ]:


# Assuming 'df' and 'impression_column_names' are defined
impression_column_names = ['rank', 'prop_id', 'is_travel_ad', 'review_rating', 'review_count', 'star_rating', 'is_free_cancellation', 'is_drr', 'price_bucket', 'num_clicks', 'is_trans']
base_columns = list(df.columns)
exploded_column_names = base_columns + impression_column_names

# Define the chunk size based on total number of rows divided by 3
chunk_size = len(df) // 6

# Function to process each chunk
def process_data(df_chunk):
    exploded_rows = []
    for index, row in tqdm(df_chunk.iterrows(), total=len(df_chunk), desc="Splitting impressions", unit="row"):
        impressions = row['impressions'].split('|')
        for impression in impressions:
            impression_data = impression.split(',')
            full_row_data = list(row.values) + impression_data
            if len(full_row_data) != len(exploded_column_names):
                print(f"Data length mismatch at index {index}: expected {len(exploded_column_names)}, got {len(full_row_data)}")
            else:
                exploded_rows.append(full_row_data)
    if exploded_rows and len(exploded_rows[0]) == len(exploded_column_names):
        return pd.DataFrame(exploded_rows, columns=exploded_column_names)
    else:
        print("Error: Data length mismatch detected, DataFrame not created.")
        return None

# Sequentially process each chunk, save, and clean up
for i in range(6):
    # Select the chunk based on the current iteration
    df_chunk = df.iloc[i * chunk_size: (i + 1) * chunk_size].copy()

    # Process the chunk
    exploded_df = process_data(df_chunk)

    # Save the processed chunk if no errors occurred
    if exploded_df is not None:
        exploded_df.to_parquet(f'../expedia_data/processed/MillionRows_Impression_{i + 1}.parquet', index=False)
    
    # Clean up processed DataFrame to free up memory
    del df_chunk, exploded_df
    gc.collect()

# Optionally, drop the remaining part of df if it won't be used further
del df
gc.collect()


In [ ]:
import pandas as pd
import gc
from tqdm import tqdm

# Function to load, append, and clean up dataframes, including dropping the 'impressions' column
def load_and_append(file_paths):
    appended_df = pd.DataFrame()  # Initialize an empty dataframe for appending
    for file_path in tqdm(file_paths, desc='Loading and Appending Files'):
        # Load the dataframe from a Parquet file
        temp_df = pd.read_parquet(file_path)
        
        # Drop the 'impressions' column if it exists
        if 'impressions' in temp_df.columns:
            temp_df.drop(columns=['impressions'], inplace=True)
        
        # Append it to the main dataframe
        appended_df = pd.concat([appended_df, temp_df], ignore_index=True)
        
        # Clear the temporary dataframe from memory
        del temp_df
        gc.collect()
    
    return appended_df

# Define the paths to the files based on how many chunks and their naming conventions
file_paths = [f'../expedia_data/processed/MillionRows_Impression_{i}.parquet' for i in range(1, 7)]

# Process the loading and appending with tqdm for the first three files
appended_df_1_to_3 = load_and_append(file_paths[:3])
# Save the combined dataframe of the first three chunks
appended_df_1_to_3.to_parquet('../expedia_data/processed/MillionRows_Impression_Combined_1_to_3.parquet', index=False)
del appended_df_1_to_3
gc.collect()

# Process the loading and appending with tqdm for the last three files
appended_df_4_to_6 = load_and_append(file_paths[3:])
# Save the combined dataframe of the last three chunks
appended_df_4_to_6.to_parquet('../expedia_data/processed/MillionRows_Impression_Combined_4_to_6.parquet', index=False)
del appended_df_4_to_6
gc.collect()


In [ ]:
gc.collect()

AllRowsImpression_df = pd.read_parquet('../expedia_data/processed/AllRows/Impression_Combined_1_to_3.parquet')
df4_to_6 = pd.read_parquet('../expedia_data/processed/AllRows/Impression_Combined_4_to_6.parquet')

AllRowsImpression_df = pd.concat([AllRowsImpression_df, df4_to_6], ignore_index=True)

AllRowsImpression_df.to_parquet('path/processed/AllRows/AllRowsImpression.parquet', index=False)

del df4_to_6, AllRowsImpression_df
gc.collect()

In [5]:
df1=pd.read_parquet('path/processed/AllRows/AllRowsImpression.parquet')

In [6]:
df1.shape

(172009471, 26)

In [7]:
df_searchid_grouped = df1.groupby('search_id', as_index=False).agg({'num_clicks':'sum', 'is_trans':'sum'})

searchid_tokeep = df_searchid_grouped[df_searchid_grouped['num_clicks'] != 0]['search_id'].tolist()  #filters out rows in df_m where the sum of 'num_clicks' isn't zero, then extracts the corresponding 'search_id' values into a list.
df1 = df1[df1['search_id'].isin(searchid_tokeep)]

# Grouping the DataFrame by 'search_id' and filtering out groups with only one row
df1 = df1.groupby('search_id').filter(lambda x: len(x) > 1)

columns_converted = ['rank', 'is_travel_ad', 'review_rating', 'review_count', 'star_rating',
                     'is_free_cancellation', 'is_drr', 'price_bucket', 'num_clicks', 'is_trans']

# Use tqdm to track progress across columns
for column in tqdm(columns_converted, desc="Converting columns"):
    df1[column] = pd.to_numeric(df1[column], errors='coerce')

searchid_withtrans_nobooking = df1[((df1['is_trans'] > 0) & (df1['num_clicks'] == 0))]['search_id'].tolist()
# Filter 'df1' to remove rows with 'search_id's in 'searchid_withtrans_nobooking'
df1 = df1[~df1['search_id'].isin(searchid_withtrans_nobooking)]


Converting columns: 100%|███████████████████████████████████████████████████████████████| 10/10 [04:02<00:00, 24.24s/it]


In [ ]:
# Assuming df1 is your DataFrame
df_m3 = df1[df1['is_travel_ad'] == 0].copy()

df_m3['rank_noad'] = df_m3.groupby('search_id').cumcount() + 1

# Create a dictionary to map search_id to rank_noad
rank_noad_map = df_m3.set_index('search_id')['rank_noad'].to_dict()

# Initialize rank_noad column with 0
df1['rank_noad'] = 0

# Update rank_noad column using the map
for search_id, rank_noad in tqdm(rank_noad_map.items(), total=len(rank_noad_map)):
    df1.loc[df1['search_id'] == search_id, 'rank_noad'] = rank_noad

In [ ]:
df1.columns

In [ ]:
df_amenities['prop_id'] = df_amenities['prop_id'].astype(int)
df1['prop_id'] = df1['prop_id'].astype(int)

df_merged = df1.merge(df_amenities, on='prop_id', how='left')

for col in df_amenities.columns:
    if col in df_merged.columns:
        df_merged[col] = df_merged[col]


# Clip values of 'star_rating' column to a range of 0 to 5
df_merged['star_rating'] = df_merged['star_rating'].clip(upper=5)
# Convert 'review_count' to numeric type
df_merged['review_count'] = pd.to_numeric(df_merged['review_count'], errors='coerce')

# Drop rows where 'review_count' is NA
df_merged.dropna(subset=['review_count'], inplace=True)

# Fill NA values based on conditions
df_merged.loc[df_merged['review_count'] == 0, 'star_rating'] = df_merged.loc[df_merged['review_count'] == 0, 'star_rating'].fillna('Not Available')
df_merged.loc[(df_merged['review_count'] > 0) & (df_merged['star_rating'].isna()), 'star_rating'] = 0

# Fill NA values based on conditions
df_merged.loc[df_merged['review_count'] == 0, 'review_rating'] = df_merged.loc[df_merged['review_count'] == 0, 'review_rating'].fillna('Not Available')
df_merged.loc[(df_merged['review_count'] > 0) & (df_merged['review_rating'].isna()), 'review_rating'] = 0

df_merged.dropna(subset=['price_bucket'], inplace=True)

df_searchid_grouped = df_merged.groupby('search_id', as_index=False).agg({'num_clicks':'sum', 'is_trans':'sum'})

searchid_tokeep = df_searchid_grouped[df_searchid_grouped['num_clicks'] != 0]['search_id'].tolist()  #filters out rows in df_m where the sum of 'num_clicks' isn't zero, then extracts the corresponding 'search_id' values into a list.
df_merged = df_merged[df_merged['search_id'].isin(searchid_tokeep)]

# Group by search_id and aggregate the sum of num_clicks and count of rows per search_id
df_searchid_grouped = df_merged.groupby('search_id', as_index=False).agg({'num_clicks':'sum', 'is_trans':'count'})

# Filter search_ids with 0 clicks or just one value
searchids_to_keep = df_searchid_grouped[(df_searchid_grouped['num_clicks'] != 0) & (df_searchid_grouped['is_trans'] > 1)]['search_id'].tolist()

# Filter df_merged based on search_ids to keep
df_filtered = df_merged[df_merged['search_id'].isin(searchids_to_keep)]

grouped_df = df_amenities.groupby('prop_id').sum()

# Calculate the number_of_amenities column as the sum of all other columns
grouped_df['number_of_amenities'] = grouped_df.sum(axis=1)

df_filtered = pd.merge(df_filtered, grouped_df[['number_of_amenities']], left_on='prop_id', right_index=True, how='left')

#df_filtered['number_of_amenities'].fillna(0, inplace=True)

df_filtered['number_of_amenities'] = df_filtered['number_of_amenities'].fillna(0)

In [ ]:
# Convert all columns except 'prop_id' to boolean dtype
df_amenities.iloc[:, 1:] = df_amenities.iloc[:, 1:].astype(bool)

# Replace True/False values with 'True'/'False'
# df_amenities.iloc[:, 1:] = df_amenities.iloc[:, 1:].replace({True: 'True', False: 'False'})

In [ ]:
amenities_columns=dict(df_amenities.dtypes)
amenities_columns

In [ ]:
df_amenities.info()

In [ ]:
exploded_df_merged = df1.drop(columns=['search_timestamp'], axis=1).merge(df_amenities, on='prop_id', how='left')


In [ ]:
exploded_df_merged.shape

In [ ]:
exploded_df_merged.isna().sum()

In [ ]:
for col in df_amenities.columns:
    if col in exploded_df_merged.columns:
        exploded_df_merged[col].fillna(0, inplace=True)


In [ ]:
exploded_df_merged.to_csv('../expedia_data/processed/merged_hundredk_rows_Impression.csv', index=False)

In [ ]:
gc.collect()

In [ ]:
exploded_df_merged.shape

In [ ]:
df2=exploded_df_merged.copy()

In [ ]:
exploded_df_merged.shape

In [ ]:
# Clip values of 'star_rating' column to a range of 0 to 5
df2['star_rating'] = df2['star_rating'].clip(upper=5)
# Convert 'review_count' to numeric type
df2['review_count'] = pd.to_numeric(df2['review_count'], errors='coerce')

# Drop rows where 'review_count' is NA
df2.dropna(subset=['review_count'], inplace=True)

# Fill NA values based on conditions
df2.loc[df2['review_count'] == 0, 'star_rating'] = df2.loc[df2['review_count'] == 0, 'star_rating'].fillna('Not Available')
df2.loc[(df2['review_count'] > 0) & (df2['star_rating'].isna()), 'star_rating'] = 0

print(exploded_df_merged[['star_rating']].value_counts(dropna=False))
print(df2[['star_rating']].value_counts(dropna=False))

In [ ]:
df2.shape

In [ ]:
df3=df2.copy()


In [ ]:
df_amenities.info()

In [ ]:
df_amenities

In [ ]:
# Fill NA values based on conditions
df3.loc[df3['review_count'] == 0, 'review_rating'] = df3.loc[df3['review_count'] == 0, 'review_rating'].fillna('Not Available')
df3.loc[(df3['review_count'] > 0) & (df3['review_rating'].isna()), 'review_rating'] = 0
print(exploded_df_merged[['review_rating']].value_counts(dropna=False))
print(df3[['review_rating']].value_counts(dropna=False))

In [ ]:
df4=df3.copy()

In [ ]:
'''
df3.loc[df3['review_count'] == 0, 'price_bucket'] = df3.loc[df3['review_count'] == 0, 'price_bucket'].fillna('Not Available')
df3.loc[(df3['review_count'] > 0) & (df3['price_bucket'].isna()), 'price_bucket'] = 0
print(exploded_df_top50dest_searched[['price_bucket']].value_counts(dropna=False))
print(df3[['price_bucket']].value_counts(dropna=False))
'''
df4.dropna(subset=['price_bucket'], inplace=True)


In [ ]:
df4.shape

In [ ]:
df5=df4.copy()

In [ ]:
df_searchid_grouped = df5.groupby('search_id', as_index=False).agg({'num_clicks':'sum', 'is_trans':'sum'})

searchid_tokeep = df_searchid_grouped[df_searchid_grouped['num_clicks'] != 0]['search_id'].tolist()  #filters out rows in df_m where the sum of 'num_clicks' isn't zero, then extracts the corresponding 'search_id' values into a list.
df5 = df5[df5['search_id'].isin(searchid_tokeep)]

condition = df5['rank'].astype(int) > 30
df5 = df5[~condition]

In [ ]:
# Group by search_id and aggregate the sum of num_clicks and count of rows per search_id
df_searchid_grouped = df5.groupby('search_id', as_index=False).agg({'num_clicks':'sum', 'is_trans':'count'})

# Filter search_ids with 0 clicks or just one value
searchids_to_keep = df_searchid_grouped[(df_searchid_grouped['num_clicks'] != 0) & (df_searchid_grouped['is_trans'] > 1)]['search_id'].tolist()

# Filter df5 based on search_ids to keep
df5_filtered = df5[df5['search_id'].isin(searchids_to_keep)]

# Print shape of df5
print("Shape of df5:", df5.shape)

# Print shape of df5_filtered
print("Shape of df5_filtered:", df5_filtered.shape)

In [ ]:
# Group by destination_id and aggregate using count for 'is_trans' and 'search_id'
destination_stats = df5_filtered.groupby('destination_id').agg({'is_trans': 'sum', 'search_id': 'count'})

# Rename the columns for clarity
destination_stats.rename(columns={'is_trans': 'number_of_bookings', 'search_id': 'number_of_searches'}, inplace=True)

print(destination_stats[:500])

In [ ]:
destination_stats.sort_values(by='number_of_searches', ascending=False)
top_500_destinations_searched = destination_stats.nlargest(500, 'number_of_searches').index
top_500_destinations_booked = destination_stats.nlargest(500, 'number_of_bookings').index

In [ ]:
exploded_df_top500dest_searched=df5_filtered[df5_filtered['destination_id'].isin(top_500_destinations_searched)]

searchings_per_destination = exploded_df_top500dest_searched.groupby('destination_id').size().reset_index(name='number_of_searches')
searchings_per_destination = searchings_per_destination.set_index('destination_id').sort_values(by='number_of_searches', ascending=False)
print(searchings_per_destination)



In [ ]:
exploded_df_top500dest_searched.shape

In [ ]:
exploded_df_top500dest_searched.to_csv('../expedia_data/processed/exploded_df_100Krows_Top500Destinations.csv', index=False)

In [ ]:
exploded_df_top500dest_searched

In [ ]:
gc.collect()

In [ ]:
df6.shape

In [ ]:
grouped_df = df_amenities.groupby('prop_id').sum()

# Calculate the number_of_amenities column as the sum of all other columns
grouped_df['number_of_amenities'] = grouped_df.sum(axis=1)

df7 = pd.merge(df6, grouped_df[['number_of_amenities']], left_on='prop_id', right_index=True, how='left')

In [ ]:
df7

In [ ]:
# Replace missing values in 'number_of_amenities' with 0
df7['number_of_amenities'].fillna(0, inplace=True)
